# A/B Testing Course

## Lesson 11. Traffic Splitting

### Homework

#### Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from scipy.stats import ttest_ind

#### Task 1. 

Code `SplittingService` class `add_experiment` method. 

In [91]:
from pydantic import BaseModel


class Experiment(BaseModel):
    """
    id - experiment identifier.
    buckets_count - desired number of buckets.
    conflicts - a list of experiment identifiers that cannot be run simultaneously on the same users.
    """
    id: int
    buckets_count: int
    conflicts: list = []


class SplittingService:

    def __init__(self, buckets_count):
        """Class for distributing experiments and users into buckets.
        
        :param buckets_count (int): number of buckets.
        """
        self.buckets_count = buckets_count
        self.buckets = [[] for _ in range(buckets_count)]

    def add_experiment(self, experiment):
        """Checks if an experiment can be added, and adds it if possible.

        :param experiment (Experiment): experiment parameters to be added
        :return success, buckets:
            success (boolean) - whether the experiment can be added, True if possible, False otherwise
            buckets (list[list[int]]]) - list of buckets, where each bucket contains the identifiers of experiments being conducted in it.
        """
        # YOUR_CODE_HERE
        exp_id = experiment.id
        exp_buckets = experiment.buckets_count
        exp_conflicts = experiment.conflicts
        if exp_conflicts == []:
            exp_conflicts = [0]
        
        cnt = 0
        for bucket in self.buckets:
            for exp_conflict in exp_conflicts:
                if exp_conflict in bucket:
                    cnt += 1
                    break
                    
        if self.buckets_count - cnt < exp_buckets or self.buckets_count < exp_buckets:
            success = False
        else:
            success = True
            
        if success == True:
            for bucket in range(self.buckets_count):
                for exp_conflict in exp_conflicts:
                    if exp_conflict in self.buckets[bucket]:
                        continue 
                    elif exp_buckets > 0:
                        if exp_id in self.buckets[bucket]:
                            continue
                        else:
                            self.buckets[bucket].append(exp_id)
                            exp_buckets -= 1
            
        return success, self.buckets

        
def check_correct_buckets(buckets, experiments):
    for experiment in experiments:
        buckets_with_exp = [b for b in buckets if experiment.id in b]
        assert experiment.buckets_count == len(buckets_with_exp), 'Wrong number of buckets with experiments'
        parallel_experiments = set(sum(buckets_with_exp, []))
        err_msg = 'Incompatible experiments in one bucket'
        assert len(set(experiment.conflicts) & parallel_experiments) == 0, err_msg


if __name__ == '__main__':
    experiments = [
        Experiment(id=1, buckets_count=4, conflicts=[4]),
        Experiment(id=2, buckets_count=2, conflicts=[3]),
        Experiment(id=3, buckets_count=2, conflicts=[2]),
        Experiment(id=4, buckets_count=1, conflicts=[1]),
    ]
    ideal_answers = [True, True, True, False]

    splitting_service = SplittingService(buckets_count=4)
    added_experiments = []
    for index, (experiment, ideal_answer) in enumerate(zip(experiments, ideal_answers)):
        success, buckets = splitting_service.add_experiment(experiment)
        print(success, buckets)
        assert success == ideal_answer, 'The split system is functioning suboptimally or incorrectly.'
        if success:
            added_experiments.append(experiment)
        check_correct_buckets(buckets, added_experiments)
    print('simple test passed')

True [[1], [1], [1], [1]]
True [[1, 2], [1, 2], [1], [1]]
True [[1, 2], [1, 2], [1, 3], [1, 3]]
False [[1, 2], [1, 2], [1, 3], [1, 3]]
simple test passed


#### Task 2. 

tbc..